In [1]:
import torch
import torch.nn.functional as F

def conv2d(feature, kernel, stride=1, padding=0):
    # 获取输入特征图的维度信息
    batch_size, C, H, W = feature.shape
    # 获取卷积核的维度信息
    C_out, C_in, K_h, K_w = kernel.shape
    # 计算输出特征图的大小, 向下取整
    H_out = (H+2*padding-K_h) // stride +1
    W_out = (W+2*padding-K_w) // stride +1

    # 判断是否需要进行填充操作
    if padding > 0:
        feature = F.pad(feature, (padding, padding, padding, padding))

    # 初始化输出
    output = torch.zeros((batch_size, C_out,  H_out, W_out))
    for i in range(H_out):
        for j in range(W_out):
            # 确定当前窗口的位置
            h_start = i * stride
            h_end = h_start + K_h
            w_start = j * stride
            w_end = w_start + K_w

            # 获取输入的局部区域后与卷积核大小进行点乘操作
            local_region = feature[:, :, h_start:h_end, w_start:w_end]
            # 卷积操作：逐元素相乘并求和
            for c_out in range(C_out):
                # 逐元素相乘并求和，获得卷积结果
                output[:, c_out, i, j] = torch.sum(local_region * kernel[c_out, :, :, :], dim=(1, 2, 3))+bias
    
    return output


# 随机初始化一个输入，大小为(B, C, H, W)
feature = torch.randn(4, 3, 5, 5)
# 随机初始化一个卷积核，大小为(C_out, C_in, H, W)
kernel = torch.randn(16, 3, 3, 3)
bias = torch.tensor([1.0])  # 偏置
output = conv2d(feature, kernel, stride=1, padding=0)
output.shape

torch.Size([4, 16, 3, 3])